In [ ]:
!pip install Sastrawi
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Approach: KMeans Clustering of texts

Most code from https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('team300_texts.xlsx')

In [ ]:
df = df.loc[df.action_enter_description.isna() == False]
#df = df.loc[df.action_enter_description.str.contains("non") == False]
df

,PID,source,source_other,source_other_recoded,action_enter_description
0,108,BeReal,NaN,NaN,friends drinking
1,164,BeReal,NaN,NaN,some random people at a party
2,108,BeReal,NaN,NaN,party
3,290,BeReal,NaN,NaN,wine
4,253,BeReal,NaN,NaN,some girl has her and her friends drinking as ...
...,...,...,...,...,...
6472,460,YouTube,NaN,NaN,wine
6473,460,YouTube,NaN,NaN,beer
6474,460,YouTube,NaN,NaN,it was a beer
6475,460,YouTube,NaN,NaN,beer


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    stop_words="english",
)

X_tfidf = vectorizer.fit_transform(df.action_enter_description)

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer


lsa = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False))
X_lsa = lsa.fit_transform(X_tfidf)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
clusters = 40

kmeans = KMeans(
    n_clusters=clusters,
    max_iter=100,
    n_init=5,
)

df['label'] = kmeans.fit_predict(X_lsa)

cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
print(f"Number of elements asigned to each cluster: {cluster_sizes}")

centroids = kmeans.cluster_centers_.argsort()[:, ::-1] ## Indices of largest centroids' entries in descending order
terms = vectorizer.get_feature_names_out()
for i in range(clusters):
    print("Cluster %d:" % i, end='')
    for ind in centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Number of elements asigned to each cluster: [ 96 553  60  40  84 140  28 113  15  98  89  38  54  44  73  72  51  62
  37  81  47  71 148  36  48  25  62  30  37  50  58  61 112  45  35  57
  42  49  37  24]
Cluster 0: ads 21 alcohol ad advertisement adult alcoholism alcoholic advertising background
Cluster 1: 21 background alchohol alcoholism asked audio book billboard breaking advertising
Cluster 2: adult add 21 account alchohol adults dahmer cup cool alc
Cluster 3: cousin clip crown 21 containing content coors better bunch celebrity
Cluster 4: alcoholism 21 alc alchohol advertising bad ad beer beth birthday
Cluster 5: account 21 add ads beer asked business beers car bc
Cluster 6: celebrate chat character celebrating car caption 21 boyfriend apartment billboard
Cluster 7: account ad 21 away dahmer booze band background alchohol bottle
Cluster 8: beverages big bonfire billboard birthday booze com book away beer
Cluster 9: advertisement add 21 ad bud breaking business beer asked bad
Cl

In [ ]:
for i in range(clusters):
  print(df.groupby('label')['action_enter_description'].apply(list)[i][:5])

['party', 'st Patrick’s day party', 'post of a party', 'Halloween party', 'it’s was a party and they were holding up shots']
['white claw', 'individuals posing with bottles', 'white claw on a boat', 'so I recently got Facebook and still don’t know how to use it that much and I keep getting recommended stuff even tho I never view anything on it like “users like you view this” and i usually ignore those but even though it’s aware of my age it showed an ad promoting alcohol', 'A group titled “Alcoholics For President”']
['another add', 'add', 'add', 'add', 'add']
['It was a Modern Family’s season 6 episode 19 where two characters were tricked into thinking they were drinking Tequila when it was just water. They convinced themselves they were drunk (even when they only drank water) and started to do ridiculous things.', 'Was watching a marvel movie featuring Thor!!!! He was going through a depressive episode or something due to Thanos snapping half of all living life, and was only seen ret

In [ ]:
exp = pd.DataFrame(df.groupby('label')['action_enter_description'].apply(list))
exp

,action_enter_description
label,
0,"[party, st Patrick’s day party, post of a part..."
1,"[white claw, individuals posing with bottles, ..."
2,"[another add, add, add, add, add, add, add, ad..."
3,[It was a Modern Family’s season 6 episode 19 ...
4,[I saw on the music video from pitbull wear th...
5,"[beer boxes, beer, I seen post with someone ho..."
6,"[Snoop Dogg liquor commercials, a girl was hol..."
7,"[saw ad for alcohol, ad for alcohol, alc ad, a..."
8,"[na, na, na, na, na, na, na, na, na, na, na, n..."


In [ ]:
exp.to_excel('test.xlsx', index=False)